In [ ]:
import numpy as np
import tensorly as tl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import torch
import torchvision

from torchsummary import summary

In [ ]:
resnet18 = torchvision.models.resnet18(pretrained=True)

In [ ]:
resnet18.cuda()
summary(resnet18, input_size=(3,224,224))

In [ ]:
for p in resnet18.named_parameters():
    print(p[0])

In [ ]:
Y = resnet18.layer4[1].conv1.weight.data.detach().cpu().clone().numpy()
Y = Y.reshape(512,512,9).transpose([2,0,1])
Y = np.transpose(Y, [1,2,0])

In [ ]:
Y.shape

In [ ]:
R = 50
R1, R2 = 30,30

In [ ]:
core.shape

In [ ]:
U = np.diag(np.ones(90))[:,:R1]
V = np.diag(np.ones(90))[:,:R2]
A = np.random.randn(R1,R)
B = np.random.randn(R2,R)
C = np.random.randn(9,R)

#Y_restored = np.transpose(tl.kruskal_to_tensor((np.ones(R), (U@A, V@B, C))), [0,1,2])

#print("Random tensors. Norm is", np.linalg.norm(Y-Y_restored))

In [ ]:
Y1 = tl.unfold(core,0)
for i in range(20):
    #print("Relative Norm is", np.linalg.norm((U@A @ (tl.tenalg.khatri_rao([C, V@B])).T) - Y1)/\
         # np.linalg.norm(Y1))
    A = U.T @ tl.unfold(core,0) @ (tl.tenalg.khatri_rao([C, V@B])) @ np.linalg.inv(C.T@C * (B.T@B))
    Q = tl.unfold(core,0) @ (tl.tenalg.khatri_rao([C, V@B]))@ np.linalg.inv((C.T@C * (B.T@B))) @\
        tl.tenalg.khatri_rao([C, V@B]).T @ tl.unfold(core,0).T

    eigvals, eigvecs = np.linalg.eigh(Q)
    indices = np.argsort(np.abs(eigvals))[-R1:]
    
    U = eigvecs[:,indices]
    
    B = V.T @ tl.unfold(core,1) @ (tl.tenalg.khatri_rao([C, U@A])) @ np.linalg.inv(C.T@C * (A.T@A))
    """
    R_ = tl.unfold(core,1) @ tl.tenalg.khatri_rao([C, U@A]) @ np.linalg.inv(C.T@C * (A.T@A)) @ \
        tl.tenalg.khatri_rao([C, U@A]).T@tl.unfold(core,1).T
    eigvals, eigvecs = np.linalg.eig(R_)
    indices = np.argsort(np.abs(eigvals))[-R2:]
    V = eigvecs[:,indices]
    
    C = tl.unfold(core,2) @ (tl.tenalg.khatri_rao([U@A, V@B])) @ np.linalg.inv(B.T@B * (A.T@A))
    #C[C < 0] = 0
    
    """
    
   

    
    
    print("Relative Norm is", np.linalg.norm((U@A @ (tl.tenalg.khatri_rao([C, V@B])).T) - Y1)/\
          np.linalg.norm(Y1))

# TKD-CPD

In [ ]:
from tensorly.decomposition import partial_tucker

In [ ]:
Y = resnet18.layer4[1].conv1.weight.data.detach().cpu().clone().numpy()
Y = Y.reshape(512, 512, 9)
Y = Y.transpose((2, 1, 0))

In [ ]:
Y.shape

In [ ]:
def conduct_compression(Y, r1,r2,R):
    core, factors = partial_tucker(Y, 
                                   modes=[1,2],
                                   ranks=[r1,r2],
                                   n_iter_max=200, 
                                   tol=1e-6)
    np.save(f"core_{r1}_{r2}_{R}", core)
    print("core:", core.shape)
    for i in range(len(factors)):
        np.save(f"factor_{i}_{r1}_{r2}_{R}" , factors[i])
        print("fi",factors[i].shape)
    core_weights, core_factors = tl.decomposition.parafac(core,
                             rank=R, 
                             n_iter_max=5000,
                             tol=1e-7)
    #print(type(core_factors), len(core_factors))
    np.save(f"core_weights_{r1}_{r2}_{R}", core_weights)
    #print("core weights:", core_weights.shape)
    for i in range(len(core_factors)):
        print("core fac", core_factors[i].shape)
        np.save(f"core_factor_{i}_{r1}_{r2}_{R}", core_factors[i])
        #print("core fact:", core_factors[i].shape)
    restored_core = tl.kruskal_to_tensor((core_weights, core_factors))
    #print("rest core:", restored_core.shape)
    np.linalg.norm(core-restored_core)/np.linalg.norm(core)

    restored_conv = np.zeros_like(Y)
    restored_conv = np.zeros_like(Y)
    for i in range(9):
        restored_conv[i] = tl.tucker_to_tensor((restored_core[i], factors))

    return np.linalg.norm(restored_conv-Y)/np.linalg.norm(Y)

In [ ]:
norms = {}
for R in [100]:
    norm = conduct_compression(Y, 90,80,R)
    norms[R] = norm
norms_aggr["90 90"] = norms

In [ ]:
norms = {}
for R in [100, 120, 150, 180, 210, 240, 250, 280, 300, 330, 380, 400]:
    norm = conduct_compression(Y, 90,90,R)
    norms[R] = norm
norms_aggr["90 90"] = norqms

In [ ]:
!mv *.npy ./decomp_60/

In [ ]:
for r1 in (60,):
    print()
    print("r1 is ", r1, end="")
    for r2 in (30,60,90):
        print(r2, end = " ")
        for R in [100, 120, 150, 180, 240, 250]:
            norm = conduct_compression(Y, r1, r2,R)
            norms[R] = norm
        norms_aggr[str(r1) +" "+str(r2)] = norms

In [ ]:
vals = []
for i in list(norms.keys()):
    vals.append(norms[i])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.figure(figsize=(12,6))
plt.grid()
plt.plot(list(norms.keys()),vals )

In [ ]:
def reconstruct(r1,r2,R):
    

In [ ]:
import os
os.path.exists("./decompositions/")

In [ ]:
for r1 in (30,60,90):
    for r2 in (30,60,90):
        for R in [100, 120, 150, 180, 240, 250]:
            assert os.path.exists(f"./decompositions/core_{r1}_{r2}_{R}.npy")
            print(r1,r2,R)
            for i in range(2):
                print(i)
                assert os.path.exists(f"./decompositions/factor_{i}_{r1}_{r2}_{R}.npy")
            assert os.path.exists(f"./decompositions/core_weights_{r1}_{r2}_{R}.npy")
            for i in range(3):
                assert os.path.exists(f"./decompositions/core_factor_{i}_{r1}_{r2}_{R}.npy")
"""
core = np.load(f"./decompositions/core_{r1}_{r2}_{R}.npy")
factors = []
for i in range(3):
        factors.append(np.load(f"./decompositions/factor_{i}_{r1}_{r2}_{R}.npy"))

core_weights = np.load(f"./decompositions/core_weights_{r1}_{r2}_{R}.npy")
core_factors = []
for i in range(2):
    core_factors.append(np.load(f"./decompositions/core_factor_{i}_{r1}_{r2}_{R}.npy"))
core_factors = tuple(core_factors)
restored_core = tl.kruskal_to_tensor((core_weights, core_factors))
print(restored_core.shape)
    #np.linalg.norm(core-restored_core)/np.linalg.norm(core)
restored_conv = np.zeros_like(Y)
print(restored_conv.shape)
for i in range(9):
    print(i)
    restored_conv[i] = tl.tucker_to_tensor((restored_core[i], factors))


"""           


In [ ]:
Y.shape

In [ ]:
import torch.nn as nn
class SimpleConv(nn.Module):
    def __init__(self):
        super(SimpleConv, self).__init__()
        self.conv = nn.Conv2d(in_channels=512,
                             out_channels=512,
                             kernel_size=3)
    def forward(self,x):
        return self.conv(x)

In [ ]:
conv_model = SimpleConv()

In [ ]:
x = torch.randn((64,512,8,8),)

In [ ]:
%%timeit
conv_model(x)

In [ ]:
conv_model.cuda()

In [ ]:
x = x.cuda()

In [ ]:
%%timeit
conv_model(x)

In [ ]:
class CompressedConv(nn.Module):
    def __init__(self, cin, cout, r1,r2,r, D=3):
        super(CompressedConv, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=cin,
                              out_channels=r1,
                              kernel_size=1)
        self.conv2 = nn.Conv2d(in_channels=r1,
                              out_channels=r,
                              kernel_size=1)
        self.conv3 = nn.Conv2d(in_channels=r,
                              out_channels=r,
                              kernel_size=D)
        self.conv4 = nn.Conv2d(in_channels=r,
                              out_channels=r2,
                              kernel_size=1)
        self.conv5 = nn.Conv2d(in_channels=r2,
                              out_channels=cout,
                              kernel_size=1)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        return x

In [ ]:
compressed_conv = CompressedConv(512,512,90,90,100)

In [ ]:
%%timeit
x = torch.randn((64,512,8,8),)
compressed_conv(x)

In [ ]:
import datetime as dt

In [ ]:
def CountSpeed(r1,r2,r, dev="cpu"):
    if dev=="cpu":
        compressed_conv = CompressedConv(512,512,r1,r2,r)
    x = torch.randn((64,512,8,8),)
    if dev=="cuda":
        compressed_conv = CompressedConv(512,512,r1,r2,r).cuda()
        x = x.cuda()
    start = dt.datetime.now()
    cycles = 100
    for i in range(100):
        y = compressed_conv(x)
        y = y +1
    finish = dt.datetime.now()
    total_time = (finish-start).total_seconds()/cycles
    return total_time
    

In [ ]:
CountSpeed(90,90,100)

In [ ]:
def CountSpeed_simple(r1,r2,r, dev="cpu"):
    if dev=="cpu":
        compressed_conv = SimpleConv()
    x = torch.randn((64,512,8,8),)
    if dev=="cuda":
        compressed_conv = CompressedConv(512,512,r1,r2,r).cuda()
        x = x.cuda()
    start = dt.datetime.now()
    cycles = 100
    for i in range(100):
        y = compressed_conv(x)
        y = y +1
    finish = dt.datetime.now()
    total_time = (finish-start).total_seconds()/cycles
    return total_time
    

In [ ]:
CountSpeed_simple(90,90,100)

In [ ]:
time = np.zeros((3,3,6))

In [ ]:
for i,r1 in enumerate((30, 60, 90)):
    for j,r2 in enumerate((30,60, 90)):
        for k,r in enumerate((100, 120, 150, 180, 240, 250)):
            time[i,j,k] = CountSpeed(r1,r2,r)

In [ ]:
time

In [ ]:
np.save("time_cpu", time)

In [ ]:
time_gpu = np.zeros((3,3,6))
for i,r1 in enumerate((30, 60, 90)):
    for j,r2 in enumerate((30,60, 90)):
        for k,r in enumerate((100, 120, 150, 180, 240, 250)):
            time_gpu[i,j,k] = CountSpeed(r1,r2,r, dev='cuda')

In [ ]:
np.save("time_gpu", time_gpu)

In [ ]:
np.linalg.norm(Y)

In [ ]:
import pickle
with open("decomp_60.pickle", "wb") as f:
    pickle.dump(norms_aggr, f)

In [ ]:
norms_aggr

In [ ]:
with open("filename.pickle", "rb") as f:
    a = pickle.load(f)

In [ ]:
def CompressionRate(r1,r2,R):
    s = 9*r1*r2 + 512*(r1+r2) + 9*R + R*(r1+r2)
    init = 512*512*9.
    return init/s

In [ ]:
CompressionRate(90, 90, 100)